# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [10]:
from nltk import sent_tokenize

In [27]:
text = open('./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', encoding='windows-1251').read()
sents = sent_tokenize(text)
sents[:3]

['Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения!',
 'Федор Михайлович Достоевский\n\nИгрок\n\n(Из записок молодого человека)\n\n\n\n\nГлава I\n\nНаконец я возвратился из моей двухнедельной отлучки.',
 'Наши уже три дня как были в Рулетенбурге.']

In [28]:
[len(sent) for sent in sents][:3]

[287, 133, 41]

In [29]:
list(map(len, sents))[:3]

[287, 133, 41]

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [ ]:
[sent for sent in sents if len(sent) <= 10][:3]

In [ ]:
list(filter(lambda sent: len(sent) <= 10, sents))[:3]

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [94]:
sents[0] = ' Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения! '
sents[0]

' Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения! '

In [11]:
import dask.bag as db

In [96]:
bag = db.from_sequence(sents)
bag

dask.bag<from_sequence, npartitions=103>

In [97]:
bag.map(len).mean().compute()

72.89892252437147

In [98]:
bag.take(3)

(' Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения! ',
 'Федор Михайлович Достоевский\n\nИгрок\n\n(Из записок молодого человека)\n\n\n\n\nГлава I\n\nНаконец я возвратился из моей двухнедельной отлучки.',
 'Наши уже три дня как были в Рулетенбурге.')

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [240]:
data = json.load(open('./data/addres-book.json'))
data[:2]

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '111-19-55'}]}]

In [241]:
bag = db.from_sequence(data)
bag

dask.bag<from_sequence, npartitions=8>

In [111]:
# bag.map(lambda person: person['phones']).compute()
bag.pluck('phones').compute()

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}],
 [{'phone': '232-19-56'}, {'phone': '+7 (916) 232-19-56'}],
 [{'phone': '+7 (916) 445-19-57'}],
 [{'phone': '232-19-58'}, {'phone': '+7 (916) 232-19-58'}],
 [{'phone': '+7 (916) 445-19-59'}],
 [{'phone': '232-19-50'}, {'phone': '+7 (916) 232-19-50'}],
 [{'phone': '111-19-51'}, {'phone': '+7 (916) 445-19-51'}]]

In [112]:
bag.pluck('phones').flatten().compute()

[{'phone': '232-19-55'},
 {'phone': '+7 (916) 232-19-55'},
 {'phone': '111-19-55'},
 {'phone': '232-19-56'},
 {'phone': '+7 (916) 232-19-56'},
 {'phone': '+7 (916) 445-19-57'},
 {'phone': '232-19-58'},
 {'phone': '+7 (916) 232-19-58'},
 {'phone': '+7 (916) 445-19-59'},
 {'phone': '232-19-50'},
 {'phone': '+7 (916) 232-19-50'},
 {'phone': '111-19-51'},
 {'phone': '+7 (916) 445-19-51'}]

In [170]:
def add_type(phone):
    phone['type'] = 'mobile' if phone['phone'].startswith('+7') else 'work'
    return phone

bag.pluck('phones').flatten().map(add_type).compute()

[{'phone': '232-19-55', 'type': 'work'},
 {'phone': '+7 (916) 232-19-55', 'type': 'mobile'},
 {'phone': '111-19-55', 'type': 'work'},
 {'phone': '232-19-56', 'type': 'work'},
 {'phone': '+7 (916) 232-19-56', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-57', 'type': 'mobile'},
 {'phone': '232-19-58', 'type': 'work'},
 {'phone': '+7 (916) 232-19-58', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-59', 'type': 'mobile'},
 {'phone': '232-19-50', 'type': 'work'},
 {'phone': '+7 (916) 232-19-50', 'type': 'mobile'},
 {'phone': '111-19-51', 'type': 'work'},
 {'phone': '+7 (916) 445-19-51', 'type': 'mobile'}]

In [46]:
bag.pluck('phones').flatten().map(add_type).pluck('type').frequencies().compute()

[('work', 6), ('mobile', 7)]

In [48]:
bag.pluck('phones')\
    .flatten()\
    .map(add_type)\
    .pluck('type')\
    .frequencies()\
    .compute()


[('work', 6), ('mobile', 7)]

1 опер сложение по кускам это - binop

2 соединяем ответы - combine

3 делаем с нечетными

In [47]:
def binop(total, value):
    return total + 1

def combine(left, right):
    return left + right

(bag.pluck('phones')
    .flatten()
    .map(add_type)
    .foldby('type', binop, 0, combine, 0)
    .compute())

[('work', 6), ('mobile', 7)]

## Лабораторная работа 12

In [3]:
import dask.bag as db
import json
import re
import pandas as pd
from zipfile import ZipFile
import jsonlines

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [ ]:
# распаковываем из зип файла
with ZipFile('./data/reviews_full.zip', 'r') as zip_file:
    zip_file.extractall('./data')

In [9]:
data_path = './data/reviews_full'
import os
b = db.read_text(os.path.join(data_path, 'reviews_*.json'))

b = b.map(json.loads)
b.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

In [197]:
b.npartitions

6

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [1]:
import re

In [4]:
def add_rating(b1):
    pattern = re.compile(r'_(\d+)\.json$')
    res = json.loads(b1[0])
    res.update({'rating':int(re.findall(pattern, b1[1])[0])})
    return res


b1 = db.read_text('./reviews_full/reviews_*.json', include_path=True)
b2 = b1.map(add_rating)
b2.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

In [15]:
type(b2)

dask.bag.core.Bag

In [20]:
import dask.dataframe as dd

In [23]:
def flatten(review):
    return {
        'user_id': review['user_id'],
        'recipe_id': review['recipe_id'],
        'date': review['date'],
        'review': review['review'],
        'rating': review['rating']
    }


b22 = b2.map(flatten).to_dataframe()

In [25]:
b22.head()

,user_id,recipe_id,date,review,rating
0,452355,292657,2016-05-08,WOW!!! This is the best. I have never been abl...,0
1,329304,433404,2006-06-14,This was good but the dressing needed somethin...,0
2,227932,2008187,1985-11-19,"Very good,it was a hit for my family. I used 6...",0
3,171468,270716,2019-05-21,Made for ZWT-8 Family Picks after I saw these ...,0
4,91392,1159916,1972-09-18,Very nice slaw. I especially like that it does...,0


In [28]:
b22.to_csv('data/reviews_full_result*.csv')  

['/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result0.csv',
 '/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result1.csv',
 '/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result2.csv',
 '/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result3.csv',
 '/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result4.csv',
 '/Users/yanalazareva/Documents/УНИВЕР/Программирование/python_doc/Технологии обработки БД/12_dask_bag/data/reviews_full_result5.csv']

3. Посчитайте количество отзывов в исходном датасете.

In [10]:
b.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [17]:
# filter: keep only some elements of the sequence
b4 = b2.filter(lambda review: review['date'].split('-')[0] == '2014' or review['date'].split('-')[0] == '2015')

In [18]:
b4.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, s

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [12]:
import re

In [19]:
def proc(rev):
    rev['review'] = rev['review'].lower().split()
    rev['review'] = re.sub("[^A-Za-z ]", "", str(rev['review']))
    return rev

b5 = b4.map(proc)

In [20]:
b5.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time welltoasted bread is the key  really toast it i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor i used sourdough bread  medium cheddar cheese fast  fun great idea made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic added the olives under the cheese and baked uncovered for the  minutes served with pico de gallo sour cream and avocado chunks fantastic thanks for shar

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [23]:
b5_count = b5.count().compute()
b5_count

735274

In [47]:
b4_count = b4.count().compute()
b4_count

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [24]:
b5.pluck('rating').frequencies().compute()

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

In [43]:
def binop(total, value):
    return total + 1

def combine(left, right):
    return left + right

b5.foldby('rating', binop, 0, combine, 0).compute()

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [26]:
b5.pluck('rating').mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [42]:
def binop(total, value):
    return max(total, len(value['review']))

b5.foldby(lambda x: x['rating'], binop, 0).compute()

[(0, 6546), (1, 2860), (2, 2829), (3, 3172), (4, 6546), (5, 5283)]